# Problem Set 1: Identification of CTCF Binding Sites with Convolutional Neural Nets

In this problem set, we will explore and experiment with a convolutional neural network for image classification. The data we will use for this problem set was created by Yuchun Guo, and contains approximately 5000 images generated by the read counts of CTCF binding sites. Your job will be to experiment with our convolutional net code below to classify CTCF binding sites from ETS1 sites. The raw data for this problem set can be found in *maps.chr1/*, but the code for preprocessing the data, building, and training the network will be provided below.

The figure below shows the architecture for LeNet-5, a convolutional network used to classify handwritten digits. It is very similar to the general network architecture typically used in image recognition problems. Two convolutional layers apply filters to the image to produce layers of feature maps, and those results are subsampled through pooling layers. The final layers of the network are fully connected, and lead to an output layer with one node for each of the classes the network is trying to detect. We will use a similar architecture for our network.

<img src="http://i.imgur.com/a2wAIeW.png" alt="Max Pooling Layer" width="75%">

All the code you will need to work with is provided below. For submissions, we ask that you submit a single Jupyter notebook (overwriting a copy of this one is fine) containing the completed problems and any additional data or writeup. You will be graded based on accuracy of your results and clarity of your writeup. However, since this is a class focused on biological research methods at its core, we value the readability and reproducibility of your results above performance. This means you will have to convince the graders that your results (good or bad) make sense and that you understand the material being taught. Do not be afraid to ask for help! We will be here during office hours to help you out and guide you along the process.

## Problem 1: Convolutional Network Understanding

### a. Convolutional Layer Parameters

Before we do any coding, it is important to get a high level understanding of how convolutional layers translate the original input into features for the network to learn. A convolutional layer is defined by a set of learnable filters. For example, in our data set, the images are 50x200x1 (where the 1 refers to the depth of our image - in RGB images, this would be 3) and our first convolutional layer has filters of size 5x5x1. During each forward pass, we slide this filter along the axes of a given input and compute dot products of the filter with the cross-section of the input. This produces an output layer of size 46x196x1, whose features that are then passed to the next layer of the network. Define the **receptive field** of a node as the size of the original image that contributes information to the node (so for the aforementioned filter, the receptive field is 5x5). With this information, answer the following questions:

1. The **stride** of a convolutional filter is another hyperparameter that controls how far we slide the filter along the width and height of the image. The most common value for this hyperparameter is 1, which means that we move the filters one pixel at a time. However, some networks use larger stride values to lower the size of the output. If we took our previous 5x5x1 filter and changed the stride to be 3, what would the new output size be? 
2. Another hyperparameter of convolutional filters is **zero-padding**, which, in our case, adds zeros to both ends of the input. This is often used when a desired filter does not fully slide along the width or height, or to control the spatial size of the output (most commonly used to make the output size and the raw input size exactly the same). In this case, suppose we would like to use our 5x5x1 filter but preserve the shape of the input (i.e. we want our output layer to have size 50x200x1). What is the smallest amount of zero-padding along the height and width required to make this possible? 
3. Most convolutional neural nets don't have just one convolutional layer, and our network is no different. In our network, the first convolutional layer applies a 5x5x1 filter to the image. Then, the second convolutional layer applies a second 5x5x1 filter to the output of the first layer. Suppose both layers have a stride of 1 and no zero-padding. What is the **receptive field** of a node in the output of the second convolutional layer? What does this suggest about building deep convolutional networks? Are more convolutional layers preferable? 
4. Another common layer used in convolutional nets are **pooling layers**. These layers are often inserted after a convolutional layer to reduce the output size and therefore the number of network parameters. In our network, we will use a max-pooling layer with a filter size of 2x2 and a stride of 2. This layer takes each (non-overlapping) 2x2 region of the convolutional layer and selects the highest response value among them as a representative feature, throwing away 75% of the activations. Other types of pooling layers also exist, such as average-pooling and sum-pooling layers, but these are sparingly used and often less effective than max-pooling layers, especially in image recognition problems. Why might this be the case? An example of this type of pooling layer (courtesy of the Stanford CS231n page) can be seen below. 

<img src="http://cs231n.github.io/assets/cnn/maxpool.jpeg" alt="Max Pooling Layer" width="75%">



#################


Provide your responses here



#################

Now, we will build and run the convolutional neural network. First, run the code below to load in the data and generate training and test sets. Since each input sequence is extremely large, we will only use a portion of the given data set. However, this should not affect the performance of our CNN.

In [ ]:
# imports
import gzip
import math
import matplotlib.pyplot as plt
import numpy as np
import os.path
import time
import tensorflow as tf

In [ ]:
gz_dirs = ["./maps.chr1/DNase.CTCF.chr1.data.txt.gz", 
             "./maps.chr1/DNase.ETS1.chr1.data.txt.gz"]
data_dirs = []

# unpack data gz's
for src_name in gz_dirs:
    dest_name = src_name[:-3]
    with gzip.open(src_name, 'rb') as infile:
        with open(dest_name, 'wb') as outfile:
            for line in infile:
                outfile.write(line)
    data_dirs.append(dest_name)
    

num_classes = 2
train_size = 2000
test_size = 500

train_data = []
train_labels = []
test_data = []
test_labels = []

class_num = 0
for d in data_dirs:
    data_array = []
    with open(d, 'r') as f:
        header = f.readline()[1:]
        split_header = header.split("\t")
        y_dim, x_dim = map(int, split_header)
        for line in f:
            splitline = line.split("\t")
            splitline = map(float, splitline)
            splitline = np.array(splitline)
            data_array.append(splitline)
    data_array = np.array(data_array)
    label_array = np.zeros((data_array.shape[0], num_classes))
    label_array[:, class_num] = 1

    train_data.append(data_array[:train_size])
    train_labels.append(label_array[:train_size])
    test_data.append(data_array[train_size:test_size + train_size])
    test_labels.append(label_array[train_size:test_size + train_size])

    class_num += 1

train_data = np.concatenate(train_data)
train_labels = np.concatenate(train_labels)
test_data = np.concatenate(test_data)
test_labels = np.concatenate(test_labels)

print 'Training set size: ', train_data.shape[0]
print 'Test set size: ', test_data.shape[0]

### b. Convolutional Network in TensorFlow code

Now that the data is properly loaded, we will build our convolutional net using TensorFlow below. Before you run anything, take a little time to read over the methods and variables we have provided you and answer the following questions:

1. Our code specifies parameters for two convolutional layers. What are their filter sizes, zero-padding values, and strides? (Check the TensorFlow documentation for *tf.nn.conv2d* to see what padding="SAME" means)
2. What activation function is being applied to the first fully-connected layer? What about the second?
3. How are we initializing the weight and bias matrices? Why do we initialize the weight matrices to nonzero values?
4. What is the loss function? How is it being minimized?

#################


Provide your responses here



#################

In [ ]:
# Factory methods for creating variables and layers

def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, stride=1):
    return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='SAME')

def max_pool(x, stride=2, filter_size=2):
    return tf.nn.max_pool(x, ksize=[1, filter_size, filter_size, 1],
                        strides=[1, stride, stride, 1], padding='SAME')

def cross_entropy(y, y_real):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_real))

def build_two_fc_layers(x_inp, Ws, bs):
    h_fc1 = tf.nn.relu(tf.matmul(x_inp, Ws[0]) + bs[0])
    return tf.matmul(h_fc1, Ws[1]) + bs[1]

### c. Building the Convolutional Layers

Using the methods above, implement two convolutional layers with max pooling and ReLU activations below. We have already specified the filter sizes and layer depths in the code, and you can use the default stride and pooling filter sizes in the methods above. The output of the second convolutional layer will then be flattened and passed in to the first fully-connected layer, whose shape is also left for you to fill in below.

In [ ]:
# Model hyperparameters

batch_size = 50
learning_rate = 0.01
conv1_filter_size = 5
conv1_depth = 8
conv2_filter_size = 5
conv2_depth = 16
fc_num_hidden = 128

tf.reset_default_graph()

# Model definition begins here
train_x = tf.placeholder(tf.float32, shape=(None, x_dim*y_dim))
train_y = tf.placeholder(tf.float32, shape=(None, num_classes))

x_image = tf.reshape(train_x, [-1, x_dim, y_dim, 1])

# TODO: implement two convolutional layers with appropriate weight 
# and bias shapes specified by the model hyperparameters. You should
# take in x_image as an input and output h_pool2 as the output of the
# second convolutional layer
#########################
# Your code goes here




#########################

# TODO: what are the shapes of the variables here?
conv2_feat_map_x =     # Define the x-size of the conv2 feature map
conv2_feat_map_y =     # Define the y-size of the conv2 feature map

W_fc1 = weight_variable([conv2_feat_map_x * conv2_feat_map_y * conv2_depth, fc_num_hidden])
b_fc1 = bias_variable([fc_num_hidden])

h_pool2_flat = tf.reshape(h_pool2, [-1, conv2_feat_map_x * conv2_feat_map_y * conv2_depth])

W_fc2 = weight_variable([fc_num_hidden, num_classes])
b_fc2 = bias_variable([num_classes])

y_conv = build_two_fc_layers(h_pool2_flat, [W_fc1, W_fc2], [b_fc1, b_fc2])

loss = cross_entropy(y_conv, train_y)
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(train_y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Now that you have written the convolutional layer code, try running the original network as is, without any modifications. It should take about 10-15 minutes to train the network for 1600 steps. What do you notice about the final test set accuracy? What does this suggest about the network? 

In [ ]:
start = time.time()
num_epochs = 20

batches_per_epoch = int(len(train_data)/batch_size)
num_steps = int(num_epochs * batches_per_epoch)

sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)
print 'Initializing variables...'

epoch_loss = 0
epoch_acc = 0

for step in xrange(num_steps):
    offset = (step * batch_size) % (train_data.shape[0] - batch_size)
    batch_x = train_data[offset:(offset + batch_size), :]
    batch_y = train_labels[offset:(offset + batch_size)]

    feed_dict = {train_x: batch_x, train_y: batch_y}
    _, batch_loss, batch_preds, batch_acc = sess.run([train_step, loss, 
                                                      correct_prediction, accuracy],
                                                      feed_dict=feed_dict)
    epoch_loss += batch_loss
    epoch_acc += batch_acc

    if (step % batches_per_epoch == 0):
        epoch_loss /= batches_per_epoch
        epoch_acc /= batches_per_epoch
        print ''
        print 'Avg batch loss at step %d: %f' % (step, epoch_loss)
        print 'Training accuracy: %f' % epoch_acc
        epoch_loss = 0
        epoch_acc = 0

test_acc = sess.run(accuracy, feed_dict={train_x: test_data, 
                                           train_y: test_labels})
print 'Test accuracy: %f' % test_acc

print "Training time: ", time.time() - start

#################


Provide your responses here



#################

## Problem 2: Optimization

We will now explore several ways of improving the performance of our network. Test each one and discuss how your results were affected. Note that not all of these optimizations might work - each method is used for different models and data sets and not all of them might apply to our network. In this section, we are simply giving you an overview of the most common network optimization techniques.

### a. Shuffling the Training Set

Since the input size of our network is extremely large, we have to perform our stochastic gradient updates with respect to minibatches (subsets of the training set). However, when the minibatch size is small compared to the size of the entire training set, it could be the case that our minibatches are sampled or constructed poorly. For this problem, experiment with shuffling the training set after every epoch and report your results. What do you notice about your newly trained network? Why might that be the case?

#################


Provide your responses here



#################

### b. Dropout

When the training set is small and the network is large, it can cause the network to overfit. At a high level, a possible result of this is that each node of a fully-connected layer could encode all the information for a single training point, leading to poor generalizability to unseen data. Dropout is a technique for addressing this problem, in which nodes and their connections are randomly removed between batches during training. In TensorFlow, this is encoded using the *tf.nn.dropout* function. Use this function to add a dropout layer after the first fully-connected layer, and modify the parameters of the layer to use different dropout probabilities < 1.0. What do you notice? 

#################


Provide your responses here



#################

### c. Weight Regularization

Another method commonly used to prevent network overfitting is weight regularization. This method adds an additional term to the loss function propotional to a function of the layer's weights, preventing them from growing too large. Two commonly used weight penalties are L2 loss (*tf.nn.l2_loss*) - which penalizes proportional to the squared norm of the weights - and L1 loss (using *tf.reduce_sum* and *tf.abs*) - which penalizes proportional to the absolute value of the weights. This has not yet been implemented inside our network, so you will have to modify the loss function to include these weight penalties. Experiment with different weight penalties and report your results. What do you notice? How does weight regularization compare to dropout?

#################


Provide your responses here



#################

## Problem 3: Visualizing the Convolutional Filters

Now that you have trained a high-performing network, we would like to understand what exactly the network is doing when it is fed any specific input. In our case, since we are working with images, we'd like to determine what features the network "sees" in each layer that allow it to perform so well. In this section, we provide some example code to visualize the result of applying filters from the first and second convolutional layers to the input images. Play around with the visualization code - perhaps try visualizing specific filters across multiple images or view the effects of the filters for incorrectly classified test results. Report your results for at least three visualization experiments. What does this tell you about the types of features our network has used to recognize CTCF binding sites? At a higher level, does the network appear to be learning reasonable features?

In [ ]:
%matplotlib inline
def plot_filter(units):
    filters = units.shape[3]
    plt.figure(1, figsize=(30,20))
    n_columns = 3
    n_rows = math.ceil(filters / n_columns) + 1
    for i in range(filters):
        plt.subplot(n_rows, n_columns, i+1)
        plt.title('Filter ' + str(i))
        plt.imshow(np.transpose(units[0,:,:,i]), interpolation="nearest", cmap="gray")

In [ ]:
# We will get the convolutional layer activations for a specific sample
test_image = test_data[:1]

# h_conv1, h_conv2 are the outputs of your convolutional layers
# ex. h_conv1 = tf.nn.relu(conv2d(x, W) + b)
img_filters1, img_filters2 = sess.run([h_conv1,h_conv2], feed_dict={train_x: test_image})

In [ ]:
# Show the original image
plt.imshow(np.transpose(np.reshape(test_image, [x_dim, y_dim])), interpolation="nearest")

In [ ]:
# Show the activations of the first convolutional filters for the first test sample
plot_filter(img_filters1)

In [ ]:
# Show the activations of the second convolutional filters for the first test sample
plot_filter(img_filters2)

#################


Provide your responses here



#################

## Problem 4 (6.874/20.490/HST.506): Make your own Architecture

For this problem, we ask that you experiment with the parameters and hyperparameters of the network to get a feel for their general effect (if any). In particular, what are the effects of:

* Increasing the stride in the convolutional layers? What about the depth?
* Making the second convolutional filter size smaller than the first? Larger?
* Adding more layers to the network?

Now try to achieve the best test set performance you can by combining the techniques discussed in the previous problem. Report the best test performance you were able to achieve and the changes that you made to achieve those results. Try to make report changes to at least three of the aspects of the network.

#################


Provide your responses here



#################